In [1]:
# Python calculation for the example and exercise from Lecture 4 notes
import numpy as np
import os
import math
import pandas as pd
import scipy.stats as stat
import scipy.interpolate
import statistics

<img src="attachment:cda85664-5f24-49cf-b6d6-af17b4e3c1ac.png" width="500">

In [3]:
mu = [0.000356, 0.000267, 0.000133, 0.000153]
sigma = [[0.00007, 0.0001, -0.000045, 0.000068],
         [0.0001, 0.0004, -0.00008, 0.000241],
         [-0.000045, -0.00008, 0.000178, -0.000118],
         [0.000068, 0.000241, -0.000118, 0.000324]
        ]
# State mean and cov matrix and weights
mu10d = np.multiply(mu, 10)
sigma10d = np.multiply(sigma, 10)
w = np.array([1000000, 1000000, 1000000, 400000])


meanL = np.inner(w, mu10d) + 2500 * (0.001 * 10) / 0.01 
varianceL = np.dot(np.dot(w, sigma10d), np.transpose(w)) + 2500 * 2500 * 0.0005 * 0.0005 * 10

var = np.abs(stat.norm.ppf(0.01, loc=meanL, scale=np.sqrt(varianceL)))
print(f"The 10-day 99% VaR using the parametric VaR model is equal to: {var:,.0f}")

The 10-day 99% VaR using the parametric VaR model is equal to: 197,746


<img src="attachment:88087327-b92c-4f3d-bdfe-bdcdfa6ebf04.png" width="600">

In [13]:
os.getcwd()

'/Users/ju/Projects/00_SMU/mqf_practice/QF609_Risk_Analysis'

In [41]:
file_path = "./data/"
offset = 693594

df_appl = pd.read_csv(file_path + "AAPL.csv").set_index('Date')
df_appl.index = pd.to_datetime(df_appl.index).date
df_appl = df_appl.sort_index()

df_dbs = pd.read_csv(file_path + "DBS.csv").set_index('Date')
df_dbs.index = pd.to_datetime(df_dbs.index).date
df_dbs = df_dbs.sort_index()

df_usdsgd = pd.read_csv(file_path + "USDSGD.csv").set_index('Date')
df_usdsgd.index = pd.to_datetime(df_usdsgd.index).date
df_usdsgd = df_usdsgd.sort_index()


d1 = [ d.toordinal()-693594  for d in df_appl.index.tolist()]
p1 = df_appl['Close'].tolist()
interp1 =  scipy.interpolate.interp1d(d1, p1)

d2 = [ d.toordinal()-693594  for d in df_dbs.index.tolist()]
p2 = df_dbs['Close'].tolist()
interp2 =  scipy.interpolate.interp1d(d2, p2)

d3 = [ d.toordinal()-693594  for d in df_usdsgd.index.tolist()]
p3 = df_usdsgd['Close'].tolist()
interp3 =  scipy.interpolate.interp1d(d3, p3)

dlist = list(set(d1) | set(d2) | set(d3))

numchg = len(dlist)-1
appl = [ interp1(dlist[i+1]).flat[0] / interp1(dlist[i]).flat[0] - 1.0  for i in range(numchg) ]
dbs = [ interp2(dlist[i+1]).flat[0] / interp2(dlist[i]).flat[0] - 1.0  for i in range(numchg) ]
usdsgd = [ interp3(dlist[i+1]).flat[0] / interp3(dlist[i]).flat[0] - 1.0  for i in range(numchg) ]

alldata = { 'appl' : appl,
           'dbs' : dbs,
           'usdsgd' : usdsgd }

df_all = pd.DataFrame(alldata, columns=['appl', 'dbs', 'usdsgd'])

corr_mat = df_all.corr()

appl_mean = np.average(appl)
appl_std = statistics.stdev(appl)
dbs_mean = np.average(dbs)
dbs_std = statistics.stdev(dbs)
usdsgd_mean = np.average(usdsgd)
usdsgd_std = statistics.stdev(usdsgd)

corr_mat

,appl,dbs,usdsgd
appl,1.000000,0.203249,-0.152445
dbs,0.203249,1.000000,-0.067703
usdsgd,-0.152445,-0.067703,1.000000


In [ ]:
print("====== means =======")
(appl_mean, dbs_mean, usdsgd_mean)

====== means =======


(0.0008041651892999006, 0.001640276168029794, 3.426939636855486e-05)

In [43]:
corr_mat

,appl,dbs,usdsgd
appl,1.000000,0.203249,-0.152445
dbs,0.203249,1.000000,-0.067703
usdsgd,-0.152445,-0.067703,1.000000
